## Calculations

The following sections shows the calculations that were done on every step to forecast the net income during the next 12 months.

The `contigo_a_features` and `contigo_b_features` are Python dictionaries that store all the features for Contigo A and B, respectively. By modifying theses features the user can run the whole notebook and see the changes on the net income.

In [78]:
# Contigo A loan features
contigo_a_features = {
    "loan_amt":1000, "loan_term":1, "interest_rate":0.15,
    "delinquency_rate":0.135, "initial_num_accounts":5000,
    "account_growth_mom":0.25, "op_cost_per_acct":30
}

# Contigo B loan features
contigo_b_features = {
    "loan_amt":2000, "loan_term":1, "interest_rate":0.10,
    "delinquency_rate":0.08, "initial_num_accounts":10000,
    "account_growth_mom":0.10, "op_cost_per_acct":30
}

Where:

- **loan_amt**: The loan amount in mexican pesos.
- **loan_term**: The number of months to pay the loan amount in full.
- **interest_rate**: The monthly interest rate.
- **delinquency_rate**: The monthly estimated delinquency rate throughout the year.
- **initial_num_accounts**: The number of accounts/customers for the first month.
- **account_growth_mom**: The estimated growth rate for the accounts month-over-month.
- **op_cost_per_acct**: The current operative monthly cost per account in mexican pesos.

### Step 1: Calculating Accounts

The following function simplifies the calculations to get the total number of accounts for each month during the next 12 months. 

It uses `math.floor` to round down the accounts to the nearest whole number because the number of accounts is a **discrete variable**.

> **Note**: The goal of using this function is to reduce the amount of lines of codes.

In [79]:
def compute_accounts(initial_accounts: int, growth_rate: float, num_months=12):
    """Computes the number of accounts by each month during a period of 12 months given a MoM growth rate"""
    return [math.floor(initial_accounts * (1+ growth_rate) ** i) for i in range(num_months)]    

The total number of accounts for Contigo A and B are stored as **numpy** arrays to allow for **element-wise operations**.

In [80]:
# Number of accounts month-over-month of Contigo A
contigo_a_accounts_mom = \
    np.array(compute_accounts(contigo_a_features["initial_num_accounts"], contigo_a_features["account_growth_mom"]))

# Number of accounts month-over-month of Contigo B
contigo_b_accounts_mom = \
    np.array(compute_accounts(contigo_b_features["initial_num_accounts"], contigo_b_features["account_growth_mom"]))

### Step 2: Claculating Delinquency Accounts

The number of delinquency accounts are calculated by multiplying the number of accounts of each month by the delinquency rate.

The delinquency accounts represent the number of accounts whose payments were not made.

**Formula**:
$$DelinquencyAccounts = NumberOfAccounts \times DelinquencyRate$$

In [81]:
# Number of delinquency accounts month-over-month of Contigo A
contigo_a_delinq_accts_mom = contigo_a_accounts_mom * contigo_a_features["delinquency_rate"]

# Round down every element in the array to the nearest whole number
contigo_a_delinq_accts_mom = np.floor(contigo_a_delinq_accts_mom)

# Number of delinquency accounts month-over-month of Contigo B
contigo_b_delinq_accts_mom = contigo_b_accounts_mom * contigo_b_features["delinquency_rate"]

# Round down every element in the array to the nearest whole number
contigo_b_delinq_accts_mom = np.floor(contigo_b_delinq_accts_mom)

### Step 3: Interest Revenue

In this step a function is defined to first calculate the interest revenue **per account** (without considering the customers whose payments were not made), and then multiplies this value by each number of accounts on every month.

The interest revenue represent the amount of interests that were collected from the customers whose payments were made.

**Formula**:
$$InterestRevenuePerAccount = LoanAmount \times InterestRate \times LoanTerm \times (1 - DelinquencyRate)$$

$$TotalInterestRevenue = NumberOfAccounts \times InterestRevenuePerAccount$$

> **Note**: The goal of using this function is to reduce the amount of lines of codes.

In [82]:
def compute_interest_rev(loan_feats: dict, loan_accounts_mom):
    """Computes the interest revenue by each month during a period of 12 months"""
    interest_revenue_per_account = \
        loan_feats["loan_amt"]*loan_feats["interest_rate"]*loan_feats["loan_term"]*(1-loan_feats["delinquency_rate"])
    return loan_accounts_mom * interest_revenue_per_account

In [83]:
# Interest revenue month-over-month of Contigo A
contigo_a_interest_rev_mom = compute_interest_rev(contigo_a_features, contigo_a_accounts_mom)

# Interest revenue month-over-month of Contigo B
contigo_b_interest_rev_mom = compute_interest_rev(contigo_b_features, contigo_b_accounts_mom)

### Step 4: Delinquency Losses

The delinquency losses are calculated by multiplying the number of delinquent accounts per month by the loan amount.

The delinquency loss represents the loan amount that was uncollected during the month.

**Formula**:
$$DelinquencyLoss = DelinquentAccounts \times LoanAmount$$

In [84]:
# Delinquency loss month-over-month of Contigo A
contigo_a_delinq_loss_mom = contigo_a_delinq_accts_mom * contigo_a_features["loan_amt"]

# Delinquency loss month-over-month of Contigo B
contigo_b_delinq_loss_mom = contigo_b_delinq_accts_mom * contigo_b_features["loan_amt"]

### Step 5: Operative Cost

The operative cost is calculated by multiplying the number of accounts of each month by the operating cost of the loan product.

The operative cost for this startup represents the expenses incurred to maintain the loan accounts.

**Formula**:
$$OperativeCost = TotalAccounts \times CostPerAccount$$

In [85]:
# Operative cost month-over-month of Contigo A
contigo_a_op_cost_mom = contigo_a_accounts_mom * contigo_a_features["op_cost_per_acct"]

# Operative cost month-over-month of Contigo B
contigo_b_op_cost_mom = contigo_b_accounts_mom * contigo_b_features["op_cost_per_acct"]

### Step 6: Net Income

The net income is calculated by subtracting the total operative costs and delinquency losses from the interest collected from customers whose payments were made.

For this startup, the net income represents the amount of money that the startup has after paying all of its costs and expenses but before paying taxes.

**Formula**:

$$NetIncome = InterestRevenue - OperativeCost - DelinquencyLoss$$

In [86]:
# Net income month-over-month of Contigo A
contigo_a_net_income_mom = contigo_a_interest_rev_mom - contigo_a_op_cost_mom - contigo_a_delinq_loss_mom

# Net income month-over-month of Contigo B
contigo_b_net_income_mom = contigo_b_interest_rev_mom - contigo_b_op_cost_mom - contigo_b_delinq_loss_mom

### Step 7: Showing Results in Tabular Format

In this step all of the previous variables are used to create a table by using pandas `DataFrame`.

In [87]:
# Maps column names to values
cols_to_values = {
    "month":np.concatenate((np.arange(1,13), np.arange(1,13))),
    "loan_type":["Contigo A"] * 12 + ["Contigo B"] * 12,
    "accounts":np.concatenate((contigo_a_accounts_mom, contigo_b_accounts_mom)),
    "delinquency_accounts":np.concatenate((contigo_a_delinq_accts_mom, contigo_b_delinq_accts_mom)),
    "interest_revenue":np.concatenate((contigo_a_interest_rev_mom, contigo_b_interest_rev_mom)),
    "delinquency_loss":np.concatenate((contigo_a_delinq_loss_mom, contigo_b_delinq_loss_mom)),
    "operative_costs":np.concatenate((contigo_a_op_cost_mom, contigo_b_op_cost_mom)),
    "net_income":np.concatenate((contigo_a_net_income_mom, contigo_b_net_income_mom))
}

# Creates a DataFrame with the results of Contigo loans
income_results = pd.DataFrame(cols_to_values)

# Converts loan type column to a categorical data type
income_results["loan_type"] = pd.Categorical(income_results.loan_type)

# Converts delinquency accounts to an integer data type
income_results["delinquency_accounts"] = income_results.delinquency_accounts.astype("int")

In [88]:
income_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   month                 24 non-null     int64   
 1   loan_type             24 non-null     category
 2   accounts              24 non-null     int64   
 3   delinquency_accounts  24 non-null     int64   
 4   interest_revenue      24 non-null     float64 
 5   delinquency_loss      24 non-null     float64 
 6   operative_costs       24 non-null     int64   
 7   net_income            24 non-null     float64 
dtypes: category(1), float64(3), int64(4)
memory usage: 1.6 KB


In [89]:
income_results

,month,loan_type,accounts,delinquency_accounts,interest_revenue,delinquency_loss,operative_costs,net_income
0,1,Contigo A,5000,675,648750.00,675000.0,150000,-176250.00
1,2,Contigo A,6250,843,810937.50,843000.0,187500,-219562.50
2,3,Contigo A,7812,1054,1013607.00,1054000.0,234360,-274753.00
3,4,Contigo A,9765,1318,1267008.75,1318000.0,292950,-343941.25
4,5,Contigo A,12207,1647,1583858.25,1647000.0,366210,-429351.75
5,6,Contigo A,15258,2059,1979725.50,2059000.0,457740,-537014.50
6,7,Contigo A,19073,2574,2474721.75,2574000.0,572190,-671468.25
7,8,Contigo A,23841,3218,3093369.75,3218000.0,715230,-839860.25
8,9,Contigo A,29802,4023,3866809.50,4023000.0,894060,-1050250.50
9,10,Contigo A,37252,5029,4833447.00,5029000.0,1117560,-1313113.00


### Step 8: Exporting Results

In this step the previous table is exported to a `.csv` file in case the user needs to share the results.

In [90]:
# Export results to a CSV file
income_results.to_csv("./income_results.csv", index=False)